In [2]:
%matplotlib qt

import numpy as np
import mne
import glob
import os
mne.set_log_level('ERROR')
import matplotlib.pyplot as plt

import sys
sys.path.append('../../../bitbrain/')
from utils.csv_utils_new import *
from utils.lsl_utils_new import *
from utils.MNE_utils import *

## Buttons to annotations

In [121]:
paths = glob.glob('resting_state/*')
paths.sort()


def merge_annotations(raw, new_annotations):
    """Merge new annotations with existing ones."""
    if raw.annotations is not None:
        merged_annotations = raw.annotations + new_annotations
    else:
        merged_annotations = new_annotations
    return merged_annotations


def merge_annotations(raw, new_annotations):
    """Merge new annotations with corrected old annotations, ensuring proper alignment."""
    
    # Correct old annotations by subtracting first_samp
    old_onsets_corrected = (raw.annotations.onset - raw.first_samp / raw.info['sfreq'])

    old_annotations_corrected = mne.Annotations(
        onset=old_onsets_corrected,
        duration=raw.annotations.duration,
        description=raw.annotations.description,
        orig_time=None  # Remove absolute time reference to avoid unwanted shifts
    )

    # Merge old corrected annotations with new ones
    merged_annotations = old_annotations_corrected + new_annotations
    return merged_annotations




for path in paths: 
    raw = mne.io.read_raw(path, preload=True)
    buttons_raw = (raw[33][0][0] + raw[34][0][0] + raw[35][0][0])
    button_sum = np.sum(buttons_raw)
    
    if button_sum != 0:
        markers_loc, markers_val = buttons_to_events(
            sd_array=buttons_raw, threshold=1000, plot_button_events=False, summing_channels=False
        )

        events = np.zeros((len(markers_loc), 3))
        events[:, 0] = markers_loc
        events[:, 2] = markers_val
        events = events.astype(int)
        event_info = markers_val


        new_annotations = mne.annotations_from_events(
            events, 256, event_desc=None, first_samp=0, orig_time=None, verbose=None
        )
        
        merged_annotations = merge_annotations(raw, new_annotations)
        raw.set_annotations(merged_annotations)
        
        output_path = f'resting_state_annotated/{os.path.basename(path)}'
        raw.save(output_path, overwrite=True)
    if button_sum == 0: 
        output_path = f'resting_state_annotated/{os.path.basename(path)[:-4]}_noannotation.fif'
        raw.save(output_path, overwrite=False)



### Visually inspect markers

In [3]:
# there are only two mindfulness sessions where there are markers, so I would ignore those


paths = glob.glob('resting_state_annotated/*mindfulness-raw*')
paths.sort()
for path in paths: 
    print(path)
    raw = mne.io.read_raw(path, preload=True)
    annotations = raw.annotations
    filtered_descriptions = [desc for desc in annotations.description if not desc.startswith("BAD_")]
    for i in filtered_descriptions:
        print(i[0])


resting_state_annotated/sub0-day1-mindfulness-raw_noannotation.fif
resting_state_annotated/sub0-day2-mindfulness-raw_noannotation.fif
resting_state_annotated/sub0-day3-mindfulness-raw_noannotation.fif
resting_state_annotated/sub0-day4-mindfulness-raw_noannotation.fif
resting_state_annotated/sub1-day1-mindfulness-raw_noannotation.fif
resting_state_annotated/sub1-day2-mindfulness-raw_noannotation.fif
resting_state_annotated/sub1-day3-mindfulness-raw_noannotation.fif
resting_state_annotated/sub1-day4-mindfulness-raw_noannotation.fif
resting_state_annotated/sub2-day1-mindfulness-raw_noannotation.fif
resting_state_annotated/sub2-day2-mindfulness-raw_noannotation.fif
resting_state_annotated/sub2-day3-mindfulness-raw_noannotation.fif
resting_state_annotated/sub2-day4-mindfulness-raw_noannotation.fif
resting_state_annotated/sub3-day1-mindfulness-raw_noannotation.fif
resting_state_annotated/sub3-day2-mindfulness-raw_noannotation.fif
resting_state_annotated/sub3-day3-mindfulness-raw_noannotation

In [5]:
# there are only two mindfulness sessions where there are markers, so I would ignore those

n=39
paths = glob.glob('resting_state_annotated/*jhana*')
paths.sort()
path = paths[n]
print(path)
raw = mne.io.read_raw(path, preload=True)
for i,j in enumerate(raw.annotations):
    description = j['description'][0:3]
    print(f'{i}: {description}')
print('')

resting_state_annotated/sub5-day1-jhana-raw.fif
0: BAD
1: BAD
2: BAD
3: BAD
4: 2
5: BAD
6: 3
7: BAD
8: 4
9: BAD
10: BAD
11: BAD
12: BAD
13: BAD
14: BAD
15: BAD



## cropping the data

In [495]:
import mne
import numpy as np

# Load the raw EEG data
n = 39
paths = glob.glob('resting_state_annotated/*jhana*')
paths.sort()
for path in paths: 
    
    print(f"Loading: {path}")
    raw = mne.io.read_raw(path, preload=True)
    
    # Extract annotations
    annotations = raw.annotations
    valid_labels = {'nimitta', '1', '2', '3', '4'}  # Keep only these
    
    # Filter annotations based on labels
    valid_annotations = [(onset - raw.first_time, label) for onset, label in zip(annotations.onset, annotations.description) if label in valid_labels]
    
    # Sort by onset time to ensure correct order
    valid_annotations.sort(key=lambda x: x[0])
    
    # List to store cropped raw segments
    cropped_segments = []
    
    # Iterate through annotations and crop the raw data
    for i in range(len(valid_annotations) - 1):
        start_time = valid_annotations[i][0]
        end_time = valid_annotations[i + 1][0]  # Next annotation's start
        cropped_segment = raw.copy().crop(tmin=start_time, tmax=end_time)
        cropped_segments.append(cropped_segment)
    
    # Handle last annotation (until end of recording)
    if valid_annotations:
        start_time = valid_annotations[-1][0]
        end_time = raw.times[-1]  # Last timestamp
        cropped_segment = raw.copy().crop(tmin=start_time, tmax=end_time)
        cropped_segments.append(cropped_segment)
    
    print(f"Extracted {len(cropped_segments)} segments.")

    # saving the new segments    
    for i, segment in enumerate(cropped_segments):
        seg_des = valid_annotations[i][1][0]
        output_path = f'resting_state_byjhana/{os.path.basename(path)[:-4]}_seg_{i}_name_{seg_des}.fif'
        segment.save(output_path, overwrite=False)


Loading: resting_state_annotated/sub0-day1-jhana-raw_noannotation.fif
Extracted 0 segments.
Loading: resting_state_annotated/sub0-day2-jhana-raw_noannotation.fif
Extracted 0 segments.
Loading: resting_state_annotated/sub0-day3-jhana-raw_noannotation.fif
Extracted 0 segments.
Loading: resting_state_annotated/sub0-day4-jhana-raw_noannotation.fif
Extracted 0 segments.
Loading: resting_state_annotated/sub1-day1-jhana-raw.fif
Extracted 7 segments.
Loading: resting_state_annotated/sub1-day2-jhana-raw.fif
Extracted 3 segments.
Loading: resting_state_annotated/sub1-day3-jhana-raw.fif
Extracted 5 segments.
Loading: resting_state_annotated/sub1-day4-jhana-raw.fif
Extracted 4 segments.
Loading: resting_state_annotated/sub2-day1-jhana-raw.fif
Extracted 5 segments.
Loading: resting_state_annotated/sub2-day2-jhana-raw.fif
Extracted 5 segments.
Loading: resting_state_annotated/sub2-day3-jhana-raw.fif
Extracted 5 segments.
Loading: resting_state_annotated/sub2-day4-jhana-raw.fif
Extracted 4 segments.


Test if cropping works well

In [6]:
paths = glob.glob('resting_state_byjhana/*')
path1 = paths[40]
path2 = 'resting_state_annotated/' + os.path.basename(path1)[:-17]+'.fif'

In [7]:
raw1 = mne.io.read_raw(path1, preload=True)
raw2 = mne.io.read_raw(path2, preload=True)

raw1.plot()
raw2.plot()

print('')

2025-04-03 07:27:31.509 Python[13537:25865048] +[IMKClient subclass]: chose IMKClient_Modern
2025-04-03 07:27:31.509 Python[13537:25865048] +[IMKInputSession subclass]: chose IMKInputSession_Modern


## concatenate cropped segments

In [11]:
import os
import glob
import mne
from collections import defaultdict

# Input and output directories
input_dir = 'resting_state_byjhana'
output_dir = 'resting_state_byjhana_concatenated'
os.makedirs(output_dir, exist_ok=True)

# Gather all segmented files
paths = glob.glob(os.path.join(input_dir, '*_seg_*_name_*.fif'))
paths.sort()

# Group segments by (sub, day, condition, jhana)
grouped = defaultdict(list)
for path in paths:
    fname = os.path.basename(path)

    # Example: sub1-day3-jhana-raw_seg_2_name_2.fif
    parts = fname.split('-')
    sub = parts[0]         # sub1
    day = parts[1]         # day3
    condition = parts[2]   # jhana
    jhana_code = fname.split('name_')[-1].replace('.fif', '')  # e.g. "2"
    jhana_label = f'j{jhana_code}'

    key = (sub, day, condition, jhana_label)
    grouped[key].append(path)

# Process each group
for key, file_list in grouped.items():
    sub, day, condition, jhana_label = key
    print(f"⏳ Concatenating: {sub} {day} {condition} {jhana_label} ({len(file_list)} segments)")

    # Load and concatenate
    raws = [mne.io.read_raw_fif(p, preload=True) for p in sorted(file_list)]
    concatenated = mne.concatenate_raws(raws)

    # Save
    out_fname = f"{sub}-{day}-{condition}-{jhana_label}.fif"
    out_path = os.path.join(output_dir, out_fname)
    concatenated.save(out_path, overwrite=True)
    print(f"✅ Saved to {out_path}")


⏳ Concatenating: sub1 day1 jhana j1 (2 segments)
✅ Saved to resting_state_byjhana_concatenated/sub1-day1-jhana-j1.fif
⏳ Concatenating: sub1 day1 jhana j2 (2 segments)
✅ Saved to resting_state_byjhana_concatenated/sub1-day1-jhana-j2.fif
⏳ Concatenating: sub1 day1 jhana j3 (2 segments)
✅ Saved to resting_state_byjhana_concatenated/sub1-day1-jhana-j3.fif
⏳ Concatenating: sub1 day1 jhana j4 (1 segments)
✅ Saved to resting_state_byjhana_concatenated/sub1-day1-jhana-j4.fif
⏳ Concatenating: sub1 day2 jhana j1 (2 segments)
✅ Saved to resting_state_byjhana_concatenated/sub1-day2-jhana-j1.fif
⏳ Concatenating: sub1 day2 jhana j2 (1 segments)
✅ Saved to resting_state_byjhana_concatenated/sub1-day2-jhana-j2.fif
⏳ Concatenating: sub1 day3 jhana jn (1 segments)
✅ Saved to resting_state_byjhana_concatenated/sub1-day3-jhana-jn.fif
⏳ Concatenating: sub1 day3 jhana j1 (1 segments)
✅ Saved to resting_state_byjhana_concatenated/sub1-day3-jhana-j1.fif
⏳ Concatenating: sub1 day3 jhana j2 (1 segments)
✅ Saved